In [1]:
import zmq
import json
import mmap
import numpy as np

In [6]:
filename = "zmqdump.dat"

In [4]:
def read_headers(filename):
    with open(filename, mode='rb') as f:
        mm = mmap.mmap(f.fileno(), 0, access=mmap.ACCESS_READ)
        result = []
        try:
            index = 0
            for i in range(2):
                len_field_bytes = mm[index:index+8]
                len_field = np.frombuffer(len_field_bytes, dtype=np.int64, count=1)
                index += 8
                data = mm[index:index+len_field[0]]
                result.append(json.loads(data))
                index += len(data)
        finally:
            mm.close()
        return result

In [8]:
read_headers(filename)

[{'header_detail': 'basic', 'htype': 'dheader-1.0', 'series': 1},
 {'auto_summation': True,
  'beam_center_x': 0.0,
  'beam_center_y': 0.0,
  'bit_depth_image': 32,
  'bit_depth_readout': 16,
  'chi_increment': 0.0,
  'chi_start': 0.0,
  'compression': 'bslz4',
  'count_time': 0.0999999,
  'countrate_correction_applied': True,
  'countrate_correction_count_cutoff': 170200,
  'data_collection_date': '2022-05-10T15:36:20.175+02:00',
  'description': 'Dectris EIGER2 Si 500K',
  'detector_distance': 0.0,
  'detector_number': 'E-01-0315',
  'detector_readout_time': 1e-07,
  'detector_translation': [0.0, 0.0, 0.0],
  'eiger_fw_version': 'release-2020.2.5rc2',
  'element': '',
  'flatfield_correction_applied': True,
  'frame_count_time': 0.020019979979999998,
  'frame_period': 0.020019979979999998,
  'frame_time': 0.1,
  'kappa_increment': 0.0,
  'kappa_start': 0.0,
  'nimages': 32,
  'ntrigger': 1,
  'number_of_excluded_pixels': 5,
  'omega_increment': 0.0,
  'omega_start': 0.0,
  'phi_incre

In [3]:
context = zmq.Context()
zmq_socket = context.socket(zmq.PUSH)
HOST = '127.0.0.1'
zmq_socket.bind(f"tcp://{HOST}:5014")

<SocketContext(bind='tcp://127.0.0.1:5014')>

In [10]:
with open(filename, mode='rb') as f:
    mm = mmap.mmap(f.fileno(), 0, access=mmap.ACCESS_READ)
    try:
        index = 0
        while index < len(mm):
            len_field_bytes = mm[index:index+8]
            len_field = np.frombuffer(len_field_bytes, dtype=np.int64, count=1)
            index += 8
            data = mm[index:index+len_field[0]]
            zmq_socket.send(data)
            index += len(data)
    finally:
        mm.close()